In [2]:
import os
import sys
import requests
# %matplotlib inline
# import matplotlib.pyplot as pyplot
# from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO



In [3]:
os.environ["COMPUTER_VISION_SUBSCRIPTION_KEY"] = ""
os.environ["COMPUTER_VISION_ENDPOINT"] = ""

In [5]:
if 'COMPUTER_VISION_SUBSCRIPTION_KEY' in os.environ:
    subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
else:
    print("set the key in your environment!")
    sys.exit()

In [7]:
if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    endpoint = os.environ['COMPUTER_VISION_ENDPOINT']

In [11]:
ocr_url = endpoint# + "vision/v3.1/ocr"

In [12]:
image_location = "local"
if (image_location == "web"):
    image_url = "https://image_location/image.jpeg"
    headers = {'OCP-Apim-Subscription-Key': subscription_key}
else:
    image_path = "./drivers_license1.jpeg"
    image_data = open(image_path, "rb").read()
    headers = {'OCP-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}

In [13]:

params = {'language': 'unk', 'detectOrientation': 'true'}
data = {'url': image_url}
if (image_location == "web"):
    response = requests.post(ocr_url, headers=headers, params=params, json=data)
else:
    response = requests.post(ocr_url, headers=headers, params=params, data=image_data)

response.raise_for_status()
analysis = response.json()


HTTPError: 404 Client Error: Resource Not Found for url: https://rdivision.cognitiveservices.azure.com/?language=unk&detectOrientation=true

In [ ]:
print(analysis)

In [ ]:
line_infos = [region["lines"] for region in analysis["regions"]]
line_infos

In [ ]:
word_infos = []
for line in line_infos:
    for word_metadata in line:
        for word_info in word_metadata["words"]:
            word_infos.append(word_info)

word_infos


In [ ]:
plt.figure(figsize(5,5))
image = Image.open(BytesIO(requests.get(image_url).content))
ax = plt.imshow(image, alpha=0.5)
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0], bbox[1])
    patch = Rectangle(origin, bbox[2], bbox[3],
                      fill=False, linewidth=2, color="y")
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=20, weight="bold", va="top")
plt.axis("off")
plt.show()